In [7]:
# Import necessary libraries
import pandas as pd
from sqlalchemy import create_engine

In [8]:
# Load the dataset
data = pd.read_csv('../data/NYC_air_bnb_dataset.csv')

In [9]:
# Connect to an in-memory SQLite database
engine = create_engine('sqlite:///:memory:')

In [10]:
# Store the DataFrame in the database as a table
data.to_sql('nyc_airbnb', engine, index=False)

In [11]:
# Define your SQL queries
sql_query_1 = """
SELECT * FROM nyc_airbnb
"""
result_1 = pd.read_sql(sql_query_1, engine)
result_1.head()  # Display the result of the first query


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,None,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [32]:
result_1.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [13]:
## Query 2: Top 10 listings with the highest prices
sql_query_2 = """
SELECT name, neighbourhood_group, neighbourhood, room_type, price
FROM nyc_airbnb
ORDER BY price DESC
LIMIT 10;"""
result_2 = pd.read_sql(sql_query_2, engine)
result_2

,name,neighbourhood_group,neighbourhood,room_type,price
0,Furnished room in Astoria apartment,Queens,Astoria,Private room,10000
1,Luxury 1 bedroom apt. -stunning Manhattan views,Brooklyn,Greenpoint,Entire home/apt,10000
2,1-BR Lincoln Center,Manhattan,Upper West Side,Entire home/apt,10000
3,Spanish Harlem Apt,Manhattan,East Harlem,Entire home/apt,9999
4,"Quiet, Clean, Lit @ LES & Chinatown",Manhattan,Lower East Side,Private room,9999
5,2br - The Heart of NYC: Manhattans Lower East ...,Manhattan,Lower East Side,Entire home/apt,9999
6,Beautiful/Spacious 1 bed luxury flat-TriBeCa/Soho,Manhattan,Tribeca,Entire home/apt,8500
7,Film Location,Brooklyn,Clinton Hill,Entire home/apt,8000
8,East 72nd Townhouse by (Hidden by Airbnb),Manhattan,Upper East Side,Entire home/apt,7703
9,70' Luxury MotorYacht on the Hudson,Manhattan,Battery Park City,Entire home/apt,7500


In [14]:
## Query 3: Average price for each neighborhood within the Manhattan borough
sql_query_3 = """
SELECT neighbourhood, AVG(price) AS avg_price
FROM nyc_airbnb
WHERE neighbourhood_group = 'Manhattan'
GROUP BY neighbourhood
ORDER BY avg_price DESC;"""
result_3 = pd.read_sql(sql_query_3, engine)
result_3

,neighbourhood,avg_price
0,Tribeca,490.638418
1,Battery Park City,367.557143
2,Flatiron District,341.925000
3,NoHo,295.717949
4,SoHo,287.103352
5,Midtown,282.719094
6,West Village,267.682292
7,Greenwich Village,263.405612
8,Chelsea,249.738544
9,Theater District,248.013889


In [15]:
## Query 4: Count of listings in each borough
sql_query_4 = """
SELECT neighbourhood_group, COUNT(*) AS listing_count
FROM nyc_airbnb
GROUP BY neighbourhood_group;"""
result_4 = pd.read_sql(sql_query_4, engine)
result_4

,neighbourhood_group,listing_count
0,Bronx,1091
1,Brooklyn,20104
2,Manhattan,21661
3,Queens,5666
4,Staten Island,373


In [16]:
## Query 5: Number of listings in each neighborhood with more than 50 reviews
sql_query_5 = """
SELECT neighbourhood, COUNT(*) AS listing_count
FROM nyc_airbnb
WHERE number_of_reviews > 50
GROUP BY neighbourhood
ORDER BY listing_count DESC;"""
result_5 = pd.read_sql(sql_query_5, engine)
result_5

,neighbourhood,listing_count
0,Bedford-Stuyvesant,696
1,Williamsburg,489
2,Harlem,477
3,Bushwick,320
4,Hell's Kitchen,312
...,...,...
179,Castleton Corners,1
180,Castle Hill,1
181,Bayswater,1
182,Bath Beach,1


In [17]:
## Query 6: List of hosts with multiple listings and their counts
sql_query_6 = """
SELECT host_id, host_name, COUNT(*) AS listing_count
FROM nyc_airbnb
GROUP BY host_id, host_name
HAVING COUNT(*) > 1
ORDER BY listing_count DESC;"""
result_6 = pd.read_sql(sql_query_6, engine)
result_6

,host_id,host_name,listing_count
0,219517861,Sonder (NYC),327
1,107434423,Blueground,232
2,30283594,Kara,121
3,137358866,Kazuya,103
4,12243051,Sonder,96
...,...,...,...
5149,271282883,Terence,2
5150,272127488,Jim&Daniela,2
5151,272265577,Yetunde,2
5152,272342845,Marc,2


In [18]:
## Query 7: The number of unique room types in each neighborhood
sql_query_7 = """
SELECT neighbourhood, COUNT(DISTINCT room_type) AS unique_room_types
FROM nyc_airbnb
GROUP BY neighbourhood;"""
result_7 = pd.read_sql(sql_query_7, engine)
result_7

,neighbourhood,unique_room_types
0,Allerton,2
1,Arden Heights,2
2,Arrochar,2
3,Arverne,3
4,Astoria,3
...,...,...
216,Windsor Terrace,2
217,Woodhaven,3
218,Woodlawn,3
219,Woodrow,1


In [19]:
## Query 8: Average minimum nights required for booking in each neighborhood group
sql_query_8 = """
SELECT neighbourhood_group, AVG(minimum_nights) AS avg_min_nights
FROM nyc_airbnb
GROUP BY neighbourhood_group;"""
result_8 = pd.read_sql(sql_query_8, engine)
result_8

,neighbourhood_group,avg_min_nights
0,Bronx,4.560953
1,Brooklyn,6.056556
2,Manhattan,8.579151
3,Queens,5.181433
4,Staten Island,4.831099


In [22]:
## Query 9: List of neighborhoods with no Airbnb listings
sql_query_9 = """
SELECT DISTINCT neighbourhood_group, neighbourhood
FROM nyc_airbnb
WHERE neighbourhood NOT IN (
    SELECT DISTINCT neighbourhood
    FROM nyc_airbnb
)"""
result_9 = pd.read_sql(sql_query_9, engine)
result_9

,neighbourhood_group,neighbourhood


In [24]:
## Query 10: List of hosts with the most reviews
sql_query_10 = """
SELECT host_id, host_name, SUM(number_of_reviews) AS total_reviews
FROM nyc_airbnb
GROUP BY host_id, host_name
ORDER BY total_reviews DESC
LIMIT 10"""
result_10 = pd.read_sql(sql_query_10, engine)
result_10

,host_id,host_name,total_reviews
0,37312959,Maya,2273
1,344035,Brooklyn& Breakfast -Len-,2205
2,26432133,Danielle,2017
3,35524316,Yasu & Akiko,1971
4,40176101,Brady,1818
5,4734398,Jj,1798
6,16677326,Alex And Zeena,1355
7,6885157,Randy,1346
8,219517861,Sonder (NYC),1281
9,23591164,Angela,1269


In [25]:
## Query 11: Percentage of available listings for each room type in each borough
sql_query_11 = """
SELECT neighbourhood_group, room_type,
    COUNT(*) AS total_listings,
    AVG(availability_365) AS avg_availability,
    (COUNT(*) - AVG(availability_365)) / COUNT(*) * 100 AS percentage_available
FROM nyc_airbnb
GROUP BY neighbourhood_group, room_type;"""
result_11 = pd.read_sql(sql_query_11, engine)
result_11

,neighbourhood_group,room_type,total_listings,avg_availability,percentage_available
0,Bronx,Entire home/apt,379,158.000000,58.311346
1,Bronx,Private room,652,171.331288,73.722195
2,Bronx,Shared room,60,154.216667,-157.027778
3,Brooklyn,Entire home/apt,9559,97.205147,98.983103
4,Brooklyn,Private room,10132,99.917983,99.013838
5,Brooklyn,Shared room,413,178.007264,56.898968
6,Manhattan,Entire home/apt,13199,117.140996,99.112501
7,Manhattan,Private room,7982,101.845026,98.724066
8,Manhattan,Shared room,480,138.572917,71.130642
9,Queens,Entire home/apt,2096,132.267176,93.689543


In [39]:
## Query 12: Top 10 hosts with the highest average price for their listings
sql_query_12 = """
SELECT host_id, host_name, AVG(price) AS avg_price
FROM nyc_airbnb
GROUP BY host_id, host_name
HAVING COUNT(*) >= 10
ORDER BY avg_price DESC
LIMIT 10;"""
result_12 = pd.read_sql(sql_query_12, engine)
result_12

,host_id,host_name,avg_price
0,156158778,Sally,3091.416667
1,836168,Henry,1772.727273
2,1177497,Jessica,1350.000000
3,69545883,Chayla,828.166667
4,205031545,Red Awning,720.285714
5,96098402,Wynpoints,619.000000
6,95459395,Bluebird,577.388889
7,260425153,Park Lane,410.714286
8,220229838,Chamber Hotel,404.000000
9,173685298,Janet,398.636364


In [27]:
## Query 13: Average price by room type for hosts with multiple listings
sql_query_13 = """
SELECT host_id, host_name, room_type, AVG(price) AS avg_price
FROM nyc_airbnb
WHERE host_id IN (
    SELECT host_id
    FROM nyc_airbnb
    GROUP BY host_id
    HAVING COUNT(*) > 1
)
GROUP BY host_id, host_name, room_type
ORDER BY host_id, room_type;"""
result_13 = pd.read_sql(sql_query_13, engine)
result_13

,host_id,host_name,room_type,avg_price
0,2787,John,Private room,105.000000
1,2787,John,Shared room,79.000000
2,2845,Jennifer,Entire home/apt,225.000000
3,2845,Jennifer,Shared room,99.000000
4,2881,Loli,Private room,58.500000
...,...,...,...,...
6658,272477673,Jin,Shared room,72.333333
6659,273354185,Lee & Luffy,Entire home/apt,113.000000
6660,273354185,Lee & Luffy,Private room,38.600000
6661,273392981,Giorgio Residence,Private room,101.000000


In [28]:
## Query 14: Listings with a price higher than the neighborhood average
sql_query_14 = """
SELECT name, neighbourhood_group, neighbourhood, room_type, price
FROM nyc_airbnb n
WHERE price > (
    SELECT AVG(price)
    FROM nyc_airbnb
    WHERE neighbourhood = n.neighbourhood
)
ORDER BY price DESC;"""
result_14 = pd.read_sql(sql_query_14, engine)
result_14

,name,neighbourhood_group,neighbourhood,room_type,price
0,Furnished room in Astoria apartment,Queens,Astoria,Private room,10000
1,Luxury 1 bedroom apt. -stunning Manhattan views,Brooklyn,Greenpoint,Entire home/apt,10000
2,1-BR Lincoln Center,Manhattan,Upper West Side,Entire home/apt,10000
3,Spanish Harlem Apt,Manhattan,East Harlem,Entire home/apt,9999
4,"Quiet, Clean, Lit @ LES & Chinatown",Manhattan,Lower East Side,Private room,9999
...,...,...,...,...,...
15639,Large bedroom with 200SF outdoor patio,Bronx,Hunts Point,Private room,60
15640,The Emperors Den,Bronx,Bronxdale,Entire home/apt,60
15641,Bronx Beauty: Renovated historic rowhouse.,Bronx,Hunts Point,Private room,59
15642,"Private and cozy bedroom, for rent.",Staten Island,Bull's Head,Private room,50


In [29]:
## Query 15: Listings with the highest number of reviews in each neighborhood
sql_query_15 = """
SELECT name, neighbourhood_group, neighbourhood, room_type, number_of_reviews
FROM nyc_airbnb n
WHERE number_of_reviews = (
    SELECT MAX(number_of_reviews)
    FROM nyc_airbnb
    WHERE neighbourhood = n.neighbourhood
)
ORDER BY neighbourhood;"""
result_15 = pd.read_sql(sql_query_15, engine)
result_15

,name,neighbourhood_group,neighbourhood,room_type,number_of_reviews
0,A PRIVATE FLAT / APARTMENT- $SPECIAL$,Bronx,Allerton,Entire home/apt,271
1,Staten Island full size living space-lower level,Staten Island,Arden Heights,Entire home/apt,20
2,30 Minutes from Manhattan Apartment by South B...,Staten Island,Arrochar,Entire home/apt,52
3,Arverne By The Bay Private Room & Bathroom,Queens,Arverne,Private room,201
4,Safe cute near subway& Manhattan NY NY retro s...,Queens,Astoria,Entire home/apt,441
...,...,...,...,...,...
217,Private Garden Entry,Brooklyn,Windsor Terrace,Entire home/apt,243
218,Small Cozy Room Wifi & AC near JFK,Queens,Woodhaven,Private room,386
219,Artsy 1 bedroom Apt. 20 min to 42nd Grand Cent...,Bronx,Woodlawn,Entire home/apt,197
220,Spacious center hall colonial,Staten Island,Woodrow,Entire home/apt,0


In [30]:
## Query 16: Percentage change in minimum nights required for booking between boroughs
sql_query_16 = """
SELECT n1.neighbourhood_group AS from_borough, n2.neighbourhood_group AS to_borough,
    AVG(n2.minimum_nights) - AVG(n1.minimum_nights) AS percentage_change
FROM nyc_airbnb n1
JOIN nyc_airbnb n2 ON n1.neighbourhood = n2.neighbourhood
WHERE n1.neighbourhood_group <> n2.neighbourhood_group
GROUP BY n1.neighbourhood_group, n2.neighbourhood_group;"""
result_16 = pd.read_sql(sql_query_16, engine)
result_16

,from_borough,to_borough,percentage_change


Demonstrating Joins

In [41]:
#storing result_12 into a db so that I have something to join on 
data.to_sql('result_12', engine, index=False)

In [50]:
# Define your SQL queries for all types of joins
sql_query_inner = """
SELECT *
FROM nyc_airbnb
INNER JOIN result_12
ON nyc_airbnb.host_id = result_12.host_id;
"""

sql_query_left = """
SELECT *
FROM nyc_airbnb
LEFT JOIN result_12
ON nyc_airbnb.host_id = result_12.host_id;
"""

sql_query_right = """
SELECT *
FROM nyc_airbnb
RIGHT JOIN result_12
ON nyc_airbnb.host_id = result_12.host_id;
"""

sql_query_full_outer = """
SELECT *
FROM nyc_airbnb
FULL OUTER JOIN result_12
ON nyc_airbnb.host_id = result_12.host_id;
"""

# Execute the SQL queries and display the results
result_inner = pd.read_sql(sql_query_inner, engine)
result_left = pd.read_sql(sql_query_left, engine)
# RIGHT and FULL OUTER JOINs are not currently supported
#result_right = pd.read_sql(sql_query_right, engine)
#result_full_outer = pd.read_sql(sql_query_full_outer, engine)


In [51]:
result_inner.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,...,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,...,40.60951,-73.97622,Private room,99,1,21,2018-10-27,0.50,6,153
2,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,...,40.60755,-73.97410,Private room,79,1,17,2019-06-26,0.40,6,174
3,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,...,40.60810,-73.97541,Private room,149,1,24,2019-05-11,0.64,6,180
4,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,...,40.60951,-73.97642,Shared room,79,1,15,2018-09-29,0.43,6,180


In [52]:
result_left.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,...,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,...,40.60951,-73.97622,Private room,99,1,21,2018-10-27,0.50,6,153
2,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,...,40.60755,-73.97410,Private room,79,1,17,2019-06-26,0.40,6,174
3,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,...,40.60810,-73.97541,Private room,149,1,24,2019-05-11,0.64,6,180
4,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,...,40.60951,-73.97642,Shared room,79,1,15,2018-09-29,0.43,6,180


Demonstrating Subqueries

In [53]:
# Define your SQL query with a subquery
sql_query_subquery = """
SELECT *
FROM nyc_airbnb
WHERE id IN (SELECT id FROM result_12 WHERE number_of_reviews > 200);
"""

# Execute the SQL query and display the result
result_subquery = pd.read_sql(sql_query_subquery, engine)
result_subquery.head()


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
1,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,2,430,2019-06-24,3.47,1,220
2,7322,Chelsea Perfect,18946,Doti,Manhattan,Chelsea,40.74192,-73.99501,Private room,140,1,260,2019-07-01,2.12,1,12
3,9668,front room/double bed,32294,Ssameer Or Trip,Manhattan,Harlem,40.82245,-73.95104,Private room,50,3,242,2019-06-01,2.04,3,355
4,9783,back room/bunk beds,32294,Ssameer Or Trip,Manhattan,Harlem,40.82130,-73.95318,Private room,50,3,273,2019-07-01,2.37,3,359


Demonstrating Aggregate Funtions

In [54]:
# Define your SQL queries for aggregate functions
sql_query_count = """
SELECT COUNT(*)
FROM nyc_airbnb;
"""

sql_query_avg = """
SELECT AVG(price)
FROM nyc_airbnb;
"""

# Execute the SQL queries and display the results
result_count = pd.read_sql(sql_query_count, engine)
result_avg = pd.read_sql(sql_query_avg, engine)


In [55]:
result_count.head()


,COUNT(*)
0,48895


In [56]:
result_avg.head()


,AVG(price)
0,152.720687


Demonstrating Window Functions

In [61]:
# Define your SQL query using a window function
sql_query_window = """
SELECT *,
       RANK() OVER (ORDER BY price) AS rank
FROM result_12;
"""

# Execute the SQL query and display the result
result_window = pd.read_sql(sql_query_window, engine)
result_window


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,rank
0,18750597,"Huge Brooklyn Brownstone Living, Close to it all.",8993084,Kimberly,Brooklyn,Bedford-Stuyvesant,40.69023,-73.95428,Private room,0,4,1,2018-01-06,0.05,4,28,1
1,20333471,★Hostel Style Room | Ideal Traveling Buddies★,131697576,Anisha,Bronx,East Morrisania,40.83296,-73.88668,Private room,0,2,55,2019-06-24,2.56,4,127,1
2,20523843,"MARTIAL LOFT 3: REDEMPTION (upstairs, 2nd room)",15787004,Martial Loft,Brooklyn,Bushwick,40.69467,-73.92433,Private room,0,2,16,2019-05-18,0.71,5,0,1
3,20608117,"Sunny, Quiet Room in Greenpoint",1641537,Lauren,Brooklyn,Greenpoint,40.72462,-73.94072,Private room,0,2,12,2017-10-27,0.53,2,0,1
4,20624541,Modern apartment in the heart of Williamsburg,10132166,Aymeric,Brooklyn,Williamsburg,40.70838,-73.94645,Entire home/apt,0,5,3,2018-01-02,0.15,1,73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,9528920,"Quiet, Clean, Lit @ LES & Chinatown",3906464,Amy,Manhattan,Lower East Side,40.71355,-73.98507,Private room,9999,99,6,2016-01-01,0.14,1,83,48890
48891,31340283,2br - The Heart of NYC: Manhattans Lower East ...,4382127,Matt,Manhattan,Lower East Side,40.71980,-73.98566,Entire home/apt,9999,30,0,None,NaN,1,365,48890
48892,7003697,Furnished room in Astoria apartment,20582832,Kathrine,Queens,Astoria,40.76810,-73.91651,Private room,10000,100,2,2016-02-13,0.04,1,0,48893
48893,13894339,Luxury 1 bedroom apt. -stunning Manhattan views,5143901,Erin,Brooklyn,Greenpoint,40.73260,-73.95739,Entire home/apt,10000,5,5,2017-07-27,0.16,1,0,48893


Demonstrating Common Table Expressions

In [63]:
# Define your SQL query using a Common Table Expression (CTE)
sql_query_cte = """
WITH cte AS (
    SELECT name, price
    FROM nyc_airbnb
    WHERE price > 300
)
SELECT name, AVG(price) AS avg_price
FROM cte
GROUP BY name;
"""

# Execute the SQL query and display the result
result_cte = pd.read_sql(sql_query_cte, engine)
result_cte


,name,avg_price
0,None,400.0
1,3 bedroom loft in Williamsburg,500.0
2,Heart & Soul of Greenwich Village,850.0
3,"""Gold Coast"" 1B in West Village w/ Back Yard",320.0
4,"""In The Heights"" Luxurious triplex in historic TH",595.0
...,...,...
3326,❤️Ever best located cozy flat❤️,330.0
3327,➖PRIVATE ROOFTOP ➖ LUXURY LIVING IN NEW YORK C...,489.0
3328,⭐ Oversized 4BR Loft In Prime Location!,499.0
3329,家 Home (12 mins to NYC),306.0
